# SpaceX Exploratory Data Analysis using SQL


## Introduction
Using this Python notebook, I

1.  Understand the Spacex DataSet
2.  Load the dataset  into the corresponding table in a Db2 database
3.  Execute SQL queries to answer assignment questions 


## Overview of the DataSet

SpaceX has gained worldwide attention for a series of historic milestones. 

It is the only private company ever to return a spacecraft from low-earth orbit, which it first accomplished in December 2010.
SpaceX advertises Falcon 9 rocket launches on its website with a cost of 62 million dollars wheras other providers cost upward of 165 million dollars each, much of the savings is because Space X can reuse the first stage. 


Therefore, if I can determine if the first stage will land, I can determine the cost of a launch. 

This information can be used if an alternate company wants to bid against SpaceX for a rocket launch.

This dataset includes a record for each payload carried during a SpaceX mission into outer space.


In [1]:
!pip install sqlalchemy==1.3.9

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 60.2 MB/s eta 0:00:00:00:010:01
  Preparing metadata (setup.py) ... done
  Created wheel for sqlalchemy: filename=SQLAlchemy-1.3.9-cp37-cp37m-linux_x86_64.whl size=1159121 sha256=34d1dd075b3050677f5aabf352305c22e6b7a61804b9bb7adeac37af86b7cb9d
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/03/71/13/010faf12246f72dc76b4150e6e599d13a85b4435e06fb9e51f
Successfully built sqlalchemy
  Attempting uninstall: sqlalchemy
    Found existing installation: SQLAlchemy 1.3.24
    Uninstalling SQLAlchemy-1.3.24:
      Successfully uninstalled SQLAlchemy-1.3.24


### Connect to the database

Let us first load the SQL extension and establish a connection with the database


In [ ]:
#if working locally, uncomment.

#!pip install ipython-sql

In [3]:
%load_ext sql

In [4]:
import csv, sqlite3

con = sqlite3.connect("my_data1.db")
cur = con.cursor()

In [5]:
!pip install -q pandas==1.1.5

In [6]:
%sql sqlite:///my_data1.db

'Connected: @my_data1.db'

### Downloading the dataset

In [7]:
import pandas as pd
df = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_2/data/Spacex.csv")
df.to_sql("SPACEXTBL", con, if_exists='replace', index=False, method="multi")

/home/jupyterlab/conda/envs/python/lib/python3.7/site-packages/pandas/core/generic.py:2882: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  both result in 0.1234 being formatted as 0.12.


**Removing the blank rows from table**


In [8]:
%sql create table SPACEXTABLE as select * from SPACEXTBL where Date is not null

 * sqlite:///my_data1.db
(sqlite3.OperationalError) table SPACEXTABLE already exists
[SQL: create table SPACEXTABLE as select * from SPACEXTBL where Date is not null]
(Background on this error at: http://sqlalche.me/e/e3q8)


## SQL Queries for Information Retrieval from the Database



### Displaying the names of the unique launch sites  in the space mission


In [9]:
%sql select distinct "Launch_Site" from SPACEXTBL;

 * sqlite:///my_data1.db
Done.


Launch_Site
CCAFS LC-40
VAFB SLC-4E
KSC LC-39A
CCAFS SLC-40



### Displaying 5 records where launch sites begin with the string 'CCA' 


In [20]:
%sql select * from SPACEXTBL where "Launch_Site" like "CCA%" LIMIT 5;

 * sqlite:///my_data1.db
Done.


Date,Time (UTC),Booster_Version,Launch_Site,Payload,PAYLOAD_MASS__KG_,Orbit,Customer,Mission_Outcome,Landing_Outcome
2010-06-04,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0,LEO,SpaceX,Success,Failure (parachute)
2010-12-08,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel of Brouere cheese",0,LEO (ISS),NASA (COTS) NRO,Success,Failure (parachute)
2012-05-22,7:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2,525,LEO (ISS),NASA (COTS),Success,No attempt
2012-10-08,0:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500,LEO (ISS),NASA (CRS),Success,No attempt
2013-03-01,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677,LEO (ISS),NASA (CRS),Success,No attempt


### Displaying the total payload mass carried by boosters launched by NASA (CRS)


In [11]:
%sql select sum(PAYLOAD_MASS__KG_) from SPACEXTBL where Customer = "NASA (CRS)";

 * sqlite:///my_data1.db
Done.


sum(PAYLOAD_MASS__KG_)
45596


### Displaying the average payload mass carried by booster version F9 v1.1


In [12]:
%sql select avg(PAYLOAD_MASS__KG_) from SPACEXTBL where Booster_Version like "F9 V1.1%";

 * sqlite:///my_data1.db
Done.


avg(PAYLOAD_MASS__KG_)
2534.6666666666665


### Listing the date when the first succesful landing outcome in ground pad was acheived.

In [21]:
%sql select min(Date) as "Min Date" from SPACEXTBL where Landing_Outcome = "Success (ground pad)";

 * sqlite:///my_data1.db
Done.


Min Date
2015-12-22


### Listing the names of the boosters which have success in drone ship and have payload mass greater than 4000 but less than 6000


In [14]:
%sql select Booster_Version from SPACEXTBL where Landing_Outcome = "Success (drone ship)" and PAYLOAD_MASS__KG_ between 4001 and 5999;

 * sqlite:///my_data1.db
Done.


Booster_Version
F9 FT B1022
F9 FT B1026
F9 FT B1021.2
F9 FT B1031.2


### Listing the total number of successful and failure mission outcomes


In [15]:
%sql select Mission_Outcome, count(*) as Frequency from SPACEXTBL group by Mission_Outcome;

 * sqlite:///my_data1.db
Done.


Mission_Outcome,Frequency
Failure (in flight),1
Success,98
Success,1
Success (payload status unclear),1


### Listing the names of the booster_versions which have carried the maximum payload mass. Use a subquery


In [16]:
%sql select Booster_Version from SPACEXTBL where PAYLOAD_MASS__KG_ = (select max(PAYLOAD_MASS__KG_) from SPACEXTBL);

 * sqlite:///my_data1.db
Done.


Booster_Version
F9 B5 B1048.4
F9 B5 B1049.4
F9 B5 B1051.3
F9 B5 B1056.4
F9 B5 B1048.5
F9 B5 B1051.4
F9 B5 B1049.5
F9 B5 B1060.2
F9 B5 B1058.3
F9 B5 B1051.6


### Listing the records which will display the month names, failure landing_outcomes in drone ship ,booster versions, launch_site for the months in year 2015.

**Note: SQLLite does not support monthnames. So I use substr(Date, 6,2) as month to get the months and substr(Date,0,5)='2015' for year.**

In [17]:
%%sql 
select substr("Date", 1, 5) as Year, substr("Date", 6, 2) as Month, Landing_Outcome, Booster_Version, Launch_Site from SPACEXTBL 
    where Landing_Outcome = "Failure (drone ship)" and substr("Date", 1, 4) = "2015";

 * sqlite:///my_data1.db
Done.


Year,Month,Landing_Outcome,Booster_Version,Launch_Site
2015-,01,Failure (drone ship),F9 v1.1 B1012,CCAFS LC-40
2015-,04,Failure (drone ship),F9 v1.1 B1015,CCAFS LC-40


### Ranking the count of landing outcomes (such as Failure (drone ship) or Success (ground pad)) between the date 2010-06-04 and 2017-03-20, in descending order.


In [18]:
%%sql 
select Landing_Outcome, count(*) as Frequency from SPACEXTBL 
    where Date between "2010-06-04" and "2017-03-20"
    group by Landing_Outcome
    order by count(*) DESC;

 * sqlite:///my_data1.db
Done.


Landing_Outcome,Frequency
No attempt,10
Success (drone ship),5
Failure (drone ship),5
Success (ground pad),3
Controlled (ocean),3
Uncontrolled (ocean),2
Failure (parachute),2
Precluded (drone ship),1


## Author(s)

<a href="https://www.linkedin.com/in/mnnamchi/">Martins Nnamchi</a>